# Εργαστηριακή Άσκηση 3
### Προσαρμοσμένα φίλτρα και L-ASK
Αφού μελετήσετε το Κεφάλαιο 3 και, ειδικότερα, το Παράδειγμα 3.2 των
σημειώσεων, να αντιγράψετε τον Κώδικα 3.3 αυτού του παραδείγματος σε αρχείο με
το όνομα ask_errors.m (ο κώδικας επαναλαμβάνεται εδώ για διευκόλυνση). Να
εκτελέσετε στη συνέχεια τα ακόλουθα:


```{admonition} Live Code
Press the following button to make python code interactive. It will connect you to a kernel once it says "ready" (might take a bit, especially the first time it runs).
```

<div style="text-align: center;">
  <button title="Launch thebe" class="thebelab-button thebe-launch-button" onclick="initThebe()">Python Interactive Code</button>
</div>


#### Importing packages we will need later in Python

In [1]:
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import erfc
import ipywidgets as widgets
from IPython.display import display, clear_output
from scipy.signal import convolve
from ipywidgets import Checkbox, VBox, Button, Output
print("Libraries added successfully!")

Libraries added successfully!


## Μέρος 1: Διερεύνηση του κώδικα εξομοίωσης

(α) Να τροποποιηθεί ο κώδικας, ώστε τα L στοιχεία του διανύσματος x στην εντολή
14 να λαμβάνουν τιμές από το σύνολο {±d/2, ±3d/2, ±5d/2... }, όπου η απόσταση
d των σημείων θα δίνεται ως παράμετρος. Χρησιμοποιώντας για τη συνέχεια την
τιμή d=5, να επαληθεύσετε, με υπολογισμό και προβολή σχετικού ιστογράμματος,
ότι τα στοιχεία του διανύσματος x ακολουθούν πράγματι την ομοιόμορφη
κατανομή. Χρησιμοποιήστε k=mod(nnnnn,2)+3, όπου nnnnn το τελευταίο 5-
ψήφιο τμήμα του αριθμού μητρώου σας.

Κώδικας 3.3: Υπολογισμός πιθανότητας εσφαλμένου συμβόλου σύμφωνης ASK με ορθογωνικούς 

`````{tab} Python
````python

def ask_errors(k,M,nsamp,EbN0_db):
    
    L = 2**k
    SNR_db = EbN0_db - 10*np.log10(nsamp/2/k)
    SNR = 10 ** (SNR_db * 0.1)
    x = 2*np.floor(L*np.random.rand(M)) - L + 1
    P_x = (L*L-1) / 3
    Measured_x = np.sum(x*x)/len(x)

    y = []
    for i in range(len(x)):
        for j in range(nsamp):
            y.append(x[i])
    y = np.array(y)

    noise = np.random.normal(0, np.sqrt(Measured_x/SNR),len(y))
    y_noisy = y + noise

    y = np.reshape(y_noisy, (M, nsamp))
    matched = np.ones((nsamp,1))
    z = np.matmul(y, matched)
    z = z / nsamp
    l = np.arange(-L+1,L,2)

    z = z[:,0]
    for i in range(len(z)):
        differences = np.abs(l-z[i]) # Πίνακας με τις διαφορές του σήματος από τα επίπεδα
        m = min(differences)
        [index], = np.where(differences == m)
        z[i] = l[index]
    
    errors = 0
    for i in range(len(z)):
        if x[i] != z[i]:
            errors += 1
    
    return errors

````
`````
`````{tab} Matlab
````matlab
function errors=ask_errors(k,Nsymb,nsamp,EbNo)
% Η συνάρτηση αυτή εξομοιώνει την παραγωγή και αποκωδικοποίηση
% θορυβώδους σήματος L-ASK και μετρά τον αριθμό των εσφαλμένων συμβόλων.
% Επιστρέφει τον αριθμό των εσφαλμένων συμβόλων (στη μεταβλητή errors).
% k είναι ο αριθμός των bits/σύμβολο, επομένως L=2^k -- ο αριθμός των
% διαφορετικών πλατών
% Nsymb είναι ο αριθμός των παραγόμενων συμβόλων (μήκος ακολουθίας LASK)
% nsamp ο αριθμός των δειγμάτων ανά σύμβολο (oversampling ratio)
% EbNo είναι ο ανηγμένος σηματοθορυβικός λόγος Eb/No, σε db
%
L=2^k;
SNR=EbNo-10*log10(nsamp/2/k); % SNR ανά δείγμα σήματος
% Διάνυσμα τυχαίων ακεραίων {±1, ±3, ... ±(L-1)}. Να επαληθευθεί
x=2*floor(L*rand(1,Nsymb))-L+1;
Px=(L^2-1)/3; % θεωρητική ισχύς σήματος
sum(x.^2)/length(x); % μετρούμενη ισχύς σήματος (για επαλήθευση)
y=rectpulse(x,nsamp);
n=wgn(1,length(y),10*log10(Px)-SNR);
ynoisy=y+n; % θορυβώδες σήμα
y=reshape(ynoisy,nsamp,length(ynoisy)/nsamp);
matched=ones(1,nsamp);
z=matched*y/nsamp;
A=[-L+1:2:L-1];
for i=1:length(z)
[m,j]=min(abs(A-z(i)));
z(i)=A(j);
end
err=not(x==z);
errors=sum(err);
end
````
`````



Υπόδειξη: Να παραχθούν (με την εντολή 14) τουλάχιστον 40.000 τυχαίοι ακέραιοι, και να
χρησιμοποιηθεί η εντολή hist(x,A) για τον υπολογισμό και την προβολή του ιστογράμματος,όπου Α το διάνυσμα των L διαφορετικών τιμών αυτών των ακεραίων. Να τροποποιηθεί
κατάλληλα η εντολή 15 για τον θεωρητικό υπολογισμό της ισχύος και να γίνει
επαλήθευση με την εντολή 16. Ομοίως, να τροποποιηθεί κατάλληλα και η εντολή 23.


In [2]:
def ask_errors(k,M,nsamp,EbN0_db):
    
    L = 2**k
    SNR_db = EbN0_db - 10*np.log10(nsamp/2/k)
    SNR = 10 ** (SNR_db * 0.1)
    x = 2*np.floor(L*np.random.rand(M)) - L + 1
    P_x = (L*L-1) / 3
    Measured_x = np.sum(x*x)/len(x)

    y = []
    for i in range(len(x)):
        for j in range(nsamp):
            y.append(x[i])
    y = np.array(y)

    noise = np.random.normal(0, np.sqrt(Measured_x/SNR),len(y))
    y_noisy = y + noise

    y = np.reshape(y_noisy, (M, nsamp))
    matched = np.ones((nsamp,1))
    z = np.matmul(y, matched)
    z = z / nsamp
    l = np.arange(-L+1,L,2)

    z = z[:,0]
    for i in range(len(z)):
        differences = np.abs(l-z[i]) # Πίνακας με τις διαφορές του σήματος από τα επίπεδα
        m = min(differences)
        [index], = np.where(differences == m)
        z[i] = l[index]
    
    errors = 0
    for i in range(len(z)):
        if x[i] != z[i]:
            errors += 1
    
    return errors


In [3]:
def generate_histogram(k):
    M = 40000
    L = 2 ** k

    x = 2 * np.floor(L * np.random.rand(M)) - L + 1
    bins = np.arange(-L, L + 2, 2)
    A = np.arange(-(L - 1), L, 2)
    
    fig, ax = plt.subplots(1, 1, figsize=(14, 4))
    ax.hist(x, bins=bins, edgecolor='white', color='#1F77B4')  # Use ax.hist for compatibility with widgets
    ax.set_xticks(A)
    ax.set_xlabel("Integers")
    ax.set_ylabel("Frequency")
    ax.set_title("Histogram of array x elements")
    plt.show()

k_input = widgets.IntText(
    value=3,  # default value
    description='k:',
    continuous_update=False
)

ui = widgets.VBox([k_input])
out = widgets.interactive_output(generate_histogram, {'k': k_input})

# Display the UI components
display(ui, out)

Output()

(β) Με τιμές των παραμέτρων της συνάρτησης k=4, M=60000, nsamp=20 και
EbNo=12 να εκτελέσετε τις εντολές 11 έως 22 και, στη συνέχεια, να σχεδιάσετε το
ιστόγραμμα του z με την εντολή hist(z,200). Να επαναλάβετε το παραπάνω για
EbNo=16 και EbNo=20. Τι παρατηρείτε; Να εξηγήσετε τις διαφορές στα τρία
διαγράμματα.

In [4]:
def generate_histogram(k, EbN0_db):
    M = 60000
    nsamp = 16

    L = 2**k
    SNR_db = EbN0_db - 10*np.log10(nsamp/2/k)
    SNR = 10 ** (SNR_db * 0.1)
    x = 2*np.floor(L*np.random.rand(M)) - L + 1
    P_x = (L**2 - 1) / 3
    Measured_x = np.sum(x**2)/len(x)

    y = np.repeat(x, nsamp)

    noise = np.random.normal(0, np.sqrt(Measured_x/SNR), len(y))
    y_noisy = y + noise

    y_reshaped = np.reshape(y_noisy, (M, nsamp))
    matched = np.ones((nsamp, 1))
    z = np.matmul(y_reshaped, matched) / nsamp

    A = np.arange(-(L-1), L, 2)  # Adjusted to use inside plotting
    
    fig, ax = plt.subplots(1, 1, figsize=(14, 4))
    ax.hist(z, bins=200,  edgecolor='white', color='#1F77B4')
    ax.set_xticks(A)
    ax.set_xlabel("Integers")
    ax.set_ylabel("Frequency")
    ax.legend(["Eb/N0 = " + str(EbN0_db)])
    ax.set_title('Histogram of the Noisy Signal')
    plt.show()

k_input = widgets.IntText(
    value=4,  # default value
    description='k:',
    continuous_update=False
)

EbN0_db_slider = widgets.FloatSlider(
    value=10,  # default value
    min=0,    # minimum value
    max=20,   # maximum value
    step=0.1,   # step size
    description='Eb/N0 (dB):',
    continuous_update=False
)

ui = widgets.VBox([k_input, EbN0_db_slider])

out = widgets.interactive_output(generate_histogram, {'k': k_input, 'EbN0_db': EbN0_db_slider})

# Display the UI components
display(ui, out)

Output()

(γ) Τι κάνει η εντολή 20 και τι κάνει η εντολή 22; Τι τύπου και τι διάστασης είναι οι
μεταβλητές: x στην εντολή 14, y στην εντολή 17, y στην εντολή 20, matched, z, err
και errors; 

(δ) Να εξηγηθεί η λειτουργία του βρόχου 24-27 του Κώδικα 3.3 ως ανιχνευτή
ελάχιστης απόστασης για την L-ASK.

## Μέρος 2ο: Καμπύλες επίδοσης (BER συναρτήσει του σηματοθορυβικού λόγου)

Να επαληθεύσετε την καμπύλη του σχήματος 3.10 των σημειώσεων (για ευκολία
επαναλαμβάνεται πιο κάτω) για την L-ASK, με L=2k, k=mod(nnnnn, 2)+3, όπου nnnnn
το τελευταίο 5-ψήφιο τμήμα του αριθμού μητρώου σας. Να σχεδιάσετε τη θεωρητική
καμπύλη και να υπερθέσετε τα αποτελέσματα της εξομοίωσης (διακριτά σημεία),
όπως στο σχήμα. Να ‘διαβάσετε’ από την καμπύλη 8-ASK και να γράψετε στο αρχείο
υποβολής τις τιμές BER για Eb/No={8,14,18}db. Αντίστροφα, βρείτε την κατάλληλη
τιμή Eb/No για σύστημα 8-ASK με ρυθμό μετάδοσης R=1Mbps, έτσι ώστε ο ρυθμός
λαθών να μην υπερβαίνει την τιμή {10, 100, 1000} bps.

![lab3_1.png](../content/images/lab3_1.png)

Το παραπάνω να γίνει με δύο τρόπους:

(α) Με δικό σας κύριο πρόγραμμα,το οποίο (i) θα σχεδιάζει την καμπύλη με χρήση
της σχέσης (3.33) των σημειώσεων και την προσέγγιση BER≈Pe /log2L, και (ii) θα
καλεί κατάλληλα τη συνάρτηση ask_errors()για τον υπολογισμό των
διακριτών σημείων.

Υπόδειξη: (α) Καλέστε την ask_errors(), μία φορά για κάθε διαφορετική τιμή του
EbNo, με επαρκώς υψηλή τιμή του Μ (π.χ. 20000) και υπολογίστε τη συχνότητα
λαθών.

(β) Με τη χρήση του εργαλείου BERTOOL του MATLAB. Το εργαλείο καλείται με
πληκτρολόγηση της εντολής bertool στο παράθυρο εντολών του MATLAB. Στο
παράθυρο που θα εμφανισθεί θα πρέπει να κάνετε τις κατάλληλες, για κάθε
περίπτωση, ρυθμίσεις, όπως υποδεικνύεται στα παρακάτω πλαίσια. Όπως
εξηγείται και στο Παράδειγμα 3.2 των σημειώσεων, το bertool καλεί τη
συνάρτησή μας, ask_errors(), μέσω της συνάρτησης ask_ber_func.m
του κώδικα 3.4, στην οποία και πρέπει να τεθούν κατάλληλες τιμές για τις
παραμέτρους k, Nsymb, και nsamp. Τα αρχεία και των δύο συναρτήσεων θα
πρέπει να βρίσκονται στο current folder (ή σε folder το οποίο βρίσκεται στο
path) του MATLAB. 

![lab3_2.png](../content/images/lab3_2.png)

![lab3_3.png](../content/images/lab3_3.png)

In [5]:
M = 20000
nsamp = 16
EbN0_db = np.arange(0, 18, 1)
EbN0 = 10**(EbN0_db / 10)

# Create checkboxes for each modulation level
checkbox_4qam = Checkbox(value=True, description='4-QAM')
checkbox_8qam = Checkbox(value=False, description='8-QAM')
checkbox_16qam = Checkbox(value=False, description='16-QAM')

plot_output = Output()

def plot_selected_modulations(btn=None):
    with plot_output:
        clear_output(wait=True)
        plt.figure(figsize=(10, 7))
        
        # Define colors for each modulation level
        colors = {
            '4-QAM': ('red', 'tomato'),
            '8-QAM': ('green', 'limegreen'),
            '16-QAM': ('blue', 'dodgerblue')
        }
        
        for i, (k, checkbox) in enumerate(zip([2, 3, 4], [checkbox_4qam, checkbox_8qam, checkbox_16qam])):
            if checkbox.value:
                L = 2**k
                modulation_name = f'{L}-QAM'
                ber = np.zeros(len(EbN0_db))
                for index, eb_n0 in enumerate(EbN0_db):
                    # Simulate experimental BER using a placeholder function
                    ber[index] = ask_errors(k, M, nsamp, index) / M / np.log2(L)
                
                # Plot experimental BER as points without connecting lines, using specific colors
                plt.semilogy(EbN0_db, ber, 'o', label=f'Experimental {modulation_name}', color=colors[modulation_name][0])
                
                # Theoretical BER for comparison, with distinct line color
                ber_theoretical = (((L-1)/L) * erfc(np.sqrt(EbN0 * (3 * np.log2(L)) / (L**2 - 1)))) / k
                plt.semilogy(EbN0_db, ber_theoretical, linestyle='-', label=f'Theoretical {modulation_name}', color=colors[modulation_name][1])

        plt.grid(True, which='both')
        plt.xlabel("Eb/N0 (dB)")
        plt.ylabel("BER")
        plt.legend()
        plt.show()

# Button to update the plot
plot_button = Button(description="Plot")
plot_button.on_click(plot_selected_modulations)

# Initially plot with default settings
plot_selected_modulations()

# Displaying the widgets and the output container
display(VBox([checkbox_4qam, checkbox_8qam, checkbox_16qam, plot_button]), plot_output)

Output()

## Μέρος 3ο: Υλοποίηση με συνέλιξη - Χρήση άλλων παλμών

Να αντικατασταθεί η εντολή 17 του Κώδικα 3.3 με τις εξής εντολές

`````{tab} Python
````python
# Create a rectangular pulse (orthogonal pulse) of unit energy
h = np.ones(nsamp) / np.sqrt(np.sum(np.ones(nsamp)**2))

# Upsample x to the dense grid
y = np.zeros(len(x) * nsamp)
y[::nsamp] = x  # Place the original signal values at the start of each upsampling period

# Convolve the upsampled signal with the filter's impulse response
y = convolve(y, h, mode='full')

# Truncate the tail left by the convolution to maintain the original signal length after upsampling
y = y[:M*nsamp]

````
`````
`````{tab} Matlab
````matlab

h=ones(1,nsamp); h=h/sqrt(h*h'); % κρουστική απόκριση φίλτρου
 % πομπού (ορθογωνικός παλμός μοναδιαίας ενέργειας)
y=upsample(x,nsamp); % μετατροπή στο πυκνό πλέγμα
y=conv(y,h); % το προς εκπομπή σήμα
y=y(1:M*nsamp); % περικόπτεται η ουρά που αφήνει η συνέλιξη

````
`````

Επίσης, το θορυβώδες σήμα, ynoisy, να παραχθεί με την εντολή

`````{tab} Python
````python
def add_awgn_noise(signal, SNR_dB):
    """
    Add White Gaussian Noise (AWGN) to the signal.
    
    Parameters:
    signal (numpy.ndarray): Input signal to which noise will be added.
    SNR_dB (float): Desired signal to noise ratio in dB.
    
    Returns:
    numpy.ndarray: Signal with added AWGN noise.
    """
    # Calculate signal power and convert from dB to linear scale
    sig_avg_watts = np.mean(signal ** 2)
    sig_avg_db = 10 * np.log10(sig_avg_watts)
    noise_avg_db = sig_avg_db - SNR_dB
    noise_avg_watts = 10 ** (noise_avg_db / 10)
    
    # Generate an sample of white noise
    mean_noise = 0
    noise_volts = np.random.normal(mean_noise, np.sqrt(noise_avg_watts), len(signal))
    
    # Noise up the original signal
    y_noisy = signal + noise_volts
    return y_noisy

# Assuming y (signal) and SNR (signal-to-noise ratio in dB) are defined
# Example: SNR = 10
y_noisy = add_awgn_noise(y, SNR)

````
`````
`````{tab} Matlab
````matlab
ynoisy=awgn(y,SNR,'measured'); % θορυβώδες σήμα
````
`````

και το προσαρμοσμένο φίλτρο (εντολές 21,22) να υλοποιηθεί με συνέλιξη ως:

`````{tab} Python
````python

# Create the matched filter by reversing h
matched = h[::-1]

# Convolve the noisy signal with the matched filter
yrx = convolve(ynoisy, matched, mode='full')

# Downsample the result to simulate sampling at the end of each symbol period
# Note: In Python indexing starts from 0, so we adjust the starting index accordingly
z = yrx[nsamp-1::nsamp]

````
`````
`````{tab} Matlab
````matlab
for i=1:nsamp matched(i)=h(end-i+1); end
yrx=conv(ynoisy,matched);
z = yrx(nsamp:nsamp:M*nsamp); % Yποδειγμάτιση -- στο τέλος
% κάθε περιόδου Τ

````
`````

(η εντολή που κάνει reshape το σήμα ynoisy πρέπει, φυσικά, εδώ να καταργηθεί).

(α) Να επιβεβαιωθεί ότι ο τροποποιημένος κώδικας παράγει τα ίδια αποτελέσματα,
όπως ο Κώδικας 3.3.

(β) Αφού εκτελέσετε το σώμα της τροποποιημένης συνάρτησης ask_errors()
χωρίς την προσθήκη θορύβου (δηλ. με ynoisy=y), σχεδιάστε τμήμα των σημάτων
x, y και yrx.

Γράψτε μια σύντομη εξήγηση του δεύτερου και τρίτου σχήματος.

(γ) Αντικαταστήστε τον ορθογωνικό παλμό h με άλλον, π.χ. συνημιτονικό μιας
περιόδου: h=cos(2*pi*(1:nsamp)/nsamp); h=h/sqrt(h*h’);

Εκτελέστε και πάλι την εξομοίωση με το bertool. Ποιο είναι το συμπέρασμα για την
επίδοση του νέου συστήματος L-ASK; Αν επαναφέρουμε την εντολή matched=h για
το προσαρμοσμένο φίλτρο, θα παρατηρήσουμε ότι δεν παίρνουμε σωστά
αποτελέσματα, ειδικότερα για μικρές τιμές του nsamp (π.χ. nsamp=8). Εξηγήστε το,
αφού παρατηρήσετε τη μορφή του παλμού h στις δύο περιπτώσεις (ορθογωνικό
παλμό και ημιτονικό παλμό), σχεδιάζοντάς τον με μίσχους (stem(h)). Να
επαναλάβετε το πείραμα για nsamp=32 και 64.